In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [8]:
df = pd.read_csv('./data/data.csv', index_col = 0)
df

,Pid,DealPrice,DealQty,BidQty1,Bid1,Ask1,AskQty1,midprice
TxTime,,,,,,,,
00:38.8,3227,126.5,216,60,126.5,127.0,110,126.75
00:43.8,3227,127.0,2,60,126.5,127.0,108,126.75
00:48.8,3227,127.0,6,62,126.5,127.0,102,126.75
00:53.9,3227,127.0,7,63,126.5,127.0,102,126.75
00:58.9,3227,127.0,7,93,126.5,127.0,103,126.75
...,...,...,...,...,...,...,...,...
24:39.0,3227,130.5,1,212,130.0,130.5,22,130.25
24:44.0,3227,0.0,0,213,130.0,130.5,22,130.25
24:49.0,3227,0.0,0,213,130.0,130.5,15,130.25


# setting

In [9]:
#current smoothed mid price periods
N_b = [5, 7, 9, 11]
#the mean of the next N_a smoothed mid price periods
N_a = [1, 5, 10]
#the threshold of price movement
alpha = [0.0001, 0.0002, 0.0003]

In [14]:
df['m_b'] = df['midprice'].rolling(N_b[2]).mean()
df

,Pid,DealPrice,DealQty,BidQty1,Bid1,Ask1,AskQty1,midprice,m_b
TxTime,,,,,,,,,
00:38.8,3227,126.5,216,60,126.5,127.0,110,126.75,NaN
00:43.8,3227,127.0,2,60,126.5,127.0,108,126.75,NaN
00:48.8,3227,127.0,6,62,126.5,127.0,102,126.75,NaN
00:53.9,3227,127.0,7,63,126.5,127.0,102,126.75,NaN
00:58.9,3227,127.0,7,93,126.5,127.0,103,126.75,NaN
...,...,...,...,...,...,...,...,...,...
24:39.0,3227,130.5,1,212,130.0,130.5,22,130.25,130.25
24:44.0,3227,0.0,0,213,130.0,130.5,22,130.25,130.25
24:49.0,3227,0.0,0,213,130.0,130.5,15,130.25,130.25


In [54]:
df['m_a'] = df['m_b'].shift(-1)[::-1].rolling(N_a[1]).mean()[::-1]
df

,Pid,DealPrice,DealQty,BidQty1,Bid1,Ask1,AskQty1,midprice,m_b,m_a
TxTime,,,,,,,,,,
00:38.8,3227,126.5,216,60,126.5,127.0,110,126.75,NaN,NaN
00:43.8,3227,127.0,2,60,126.5,127.0,108,126.75,NaN,NaN
00:48.8,3227,127.0,6,62,126.5,127.0,102,126.75,NaN,NaN
00:53.9,3227,127.0,7,63,126.5,127.0,102,126.75,NaN,NaN
00:58.9,3227,127.0,7,93,126.5,127.0,103,126.75,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
24:39.0,3227,130.5,1,212,130.0,130.5,22,130.25,130.25,NaN
24:44.0,3227,0.0,0,213,130.0,130.5,22,130.25,130.25,NaN
24:49.0,3227,0.0,0,213,130.0,130.5,15,130.25,130.25,NaN


In [77]:
df['pred'] = (df['m_b'] > df['m_a'] * (1 + alpha[0])) * 1
for i in range(len(df)):
    if df['m_b'].iloc[i] > df['m_a'].iloc[i] * (1 + alpha[0]):
        df['pred'].iloc[i] = 1
    elif df['m_b'].iloc[i] < df['m_a'].iloc[i] * (1 - alpha[0]):
        df['pred'].iloc[i] = -1
    else:
        df['pred'].iloc[i] = 0

In [133]:
df.head(10)

,Pid,DealPrice,DealQty,BidQty1,Bid1,Ask1,AskQty1,midprice,m_b,m_a,pred
TxTime,,,,,,,,,,,
00:38.8,3227,126.5,216,60,126.5,127.0,110,126.75,NaN,NaN,0
00:43.8,3227,127.0,2,60,126.5,127.0,108,126.75,NaN,NaN,0
00:48.8,3227,127.0,6,62,126.5,127.0,102,126.75,NaN,NaN,0
00:53.9,3227,127.0,7,63,126.5,127.0,102,126.75,NaN,NaN,0
00:58.9,3227,127.0,7,93,126.5,127.0,103,126.75,NaN,NaN,0
01:03.9,3227,127.0,102,92,126.5,127.0,8,126.75,NaN,NaN,0
01:08.9,3227,127.0,11,142,127.0,127.5,40,127.25,NaN,NaN,0
01:13.9,3227,127.0,10,139,127.0,127.5,44,127.25,NaN,126.961111,0
01:18.9,3227,127.0,10,139,127.0,127.5,44,127.25,126.916667,126.972222,-1


In [94]:
np.sum(df['pred'] == 1)

32441

In [95]:
np.sum(df['pred'] == -1)

31687

In [96]:
np.sum(df['pred'] == 0)

188932

In [104]:
df['m_a'].isnull().sum()

12

In [129]:
df.to_csv('./data/data_1.csv',index=False)

In [130]:
df.rolling(5)

Rolling [window=5,center=False,axis=0]

In [2]:
df = pd.read_csv('./data/data_1.csv', index_col = 0)
df

,DealPrice,DealQty,BidQty1,Bid1,Ask1,AskQty1,midprice,m_b,m_a,pred
Pid,,,,,,,,,,
3227,126.5,216,60,126.5,127.0,110,126.75,NaN,NaN,0
3227,127.0,2,60,126.5,127.0,108,126.75,NaN,NaN,0
3227,127.0,6,62,126.5,127.0,102,126.75,NaN,NaN,0
3227,127.0,7,63,126.5,127.0,102,126.75,NaN,NaN,0
3227,127.0,7,93,126.5,127.0,103,126.75,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...
3227,130.5,1,212,130.0,130.5,22,130.25,130.25,NaN,0
3227,0.0,0,213,130.0,130.5,22,130.25,130.25,NaN,0
3227,0.0,0,213,130.0,130.5,15,130.25,130.25,NaN,0


In [3]:
df.head(30)

,DealPrice,DealQty,BidQty1,Bid1,Ask1,AskQty1,midprice,m_b,m_a,pred
Pid,,,,,,,,,,
3227,126.5,216,60,126.5,127.0,110,126.75,NaN,NaN,0
3227,127.0,2,60,126.5,127.0,108,126.75,NaN,NaN,0
3227,127.0,6,62,126.5,127.0,102,126.75,NaN,NaN,0
3227,127.0,7,63,126.5,127.0,102,126.75,NaN,NaN,0
3227,127.0,7,93,126.5,127.0,103,126.75,NaN,NaN,0
3227,127.0,102,92,126.5,127.0,8,126.75,NaN,NaN,0
3227,127.0,11,142,127.0,127.5,40,127.25,NaN,NaN,0
3227,127.0,10,139,127.0,127.5,44,127.25,NaN,126.961111,0
3227,127.0,10,139,127.0,127.5,44,127.25,126.916667,126.972222,-1


In [4]:
df_new = pd.read_csv('./data/data.csv', index_col = 0)
df_new

,Pid,DealPrice,DealQty,BidQty1,Bid1,Ask1,AskQty1,midprice
TxTime,,,,,,,,
00:38.8,3227,126.5,216,60,126.5,127.0,110,126.75
00:43.8,3227,127.0,2,60,126.5,127.0,108,126.75
00:48.8,3227,127.0,6,62,126.5,127.0,102,126.75
00:53.9,3227,127.0,7,63,126.5,127.0,102,126.75
00:58.9,3227,127.0,7,93,126.5,127.0,103,126.75
...,...,...,...,...,...,...,...,...
24:39.0,3227,130.5,1,212,130.0,130.5,22,130.25
24:44.0,3227,0.0,0,213,130.0,130.5,22,130.25
24:49.0,3227,0.0,0,213,130.0,130.5,15,130.25


In [6]:
df_new['m_b'] = df_new['midprice'].rolling(10).mean()
df_new

,Pid,DealPrice,DealQty,BidQty1,Bid1,Ask1,AskQty1,midprice,m_b
TxTime,,,,,,,,,
00:38.8,3227,126.5,216,60,126.5,127.0,110,126.75,NaN
00:43.8,3227,127.0,2,60,126.5,127.0,108,126.75,NaN
00:48.8,3227,127.0,6,62,126.5,127.0,102,126.75,NaN
00:53.9,3227,127.0,7,63,126.5,127.0,102,126.75,NaN
00:58.9,3227,127.0,7,93,126.5,127.0,103,126.75,NaN
...,...,...,...,...,...,...,...,...,...
24:39.0,3227,130.5,1,212,130.0,130.5,22,130.25,130.25
24:44.0,3227,0.0,0,213,130.0,130.5,22,130.25,130.25
24:49.0,3227,0.0,0,213,130.0,130.5,15,130.25,130.25


In [31]:
df_new['pred'] = df_new['m_b']
for i in range(len(df)):
    if df_new['midprice'].iloc[i] > df_new['m_b'].iloc[i] * (1 + alpha[0]):
        df_new['pred'].iloc[i] = 1
    elif df_new['midprice'].iloc[i] < df_new['m_b'].iloc[i] * (1 - alpha[0]):
        df_new['pred'].iloc[i] = -1
    else:
        df_new['pred'].iloc[i] = 0

In [32]:
df_new

,Pid,DealPrice,DealQty,BidQty1,Bid1,Ask1,AskQty1,midprice,m_b,pred
TxTime,,,,,,,,,,
00:38.8,3227,126.5,216,60,126.5,127.0,110,126.75,NaN,0.0
00:43.8,3227,127.0,2,60,126.5,127.0,108,126.75,NaN,0.0
00:48.8,3227,127.0,6,62,126.5,127.0,102,126.75,NaN,0.0
00:53.9,3227,127.0,7,63,126.5,127.0,102,126.75,NaN,0.0
00:58.9,3227,127.0,7,93,126.5,127.0,103,126.75,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...
24:39.0,3227,130.5,1,212,130.0,130.5,22,130.25,130.25,0.0
24:44.0,3227,0.0,0,213,130.0,130.5,22,130.25,130.25,0.0
24:49.0,3227,0.0,0,213,130.0,130.5,15,130.25,130.25,0.0


In [35]:
df = pd.read_csv('./data/3227.csv', index_col = 0)
df

,Pid,DealPrice,DealQty,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,...,Ask3,AskQty3,BidQty4,Bid4,Ask4,AskQty4,BidQty5,Bid5,Ask5,AskQty5
TxTime,,,,,,,,,,,,,,,,,,,,,
2019/7/1 09:00,3227,126.5,216,60,126.5,127.0,110,78,126.0,127.5,...,128.0,149,82,125.0,128.5,30,12,124.5,129.0,68
2019/7/1 09:00,3227,127.0,2,60,126.5,127.0,108,78,126.0,127.5,...,128.0,149,82,125.0,128.5,30,12,124.5,129.0,68
2019/7/1 09:00,3227,127.0,6,62,126.5,127.0,102,79,126.0,127.5,...,128.0,149,82,125.0,128.5,30,17,124.5,129.0,73
2019/7/1 09:00,3227,127.0,7,63,126.5,127.0,102,78,126.0,127.5,...,128.0,149,82,125.0,128.5,30,17,124.5,129.0,74
2019/7/1 09:00,3227,127.0,7,93,126.5,127.0,103,78,126.0,127.5,...,128.0,149,82,125.0,128.5,29,17,124.5,129.0,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019/11/29 13:24,3227,130.5,1,212,130.0,130.5,22,142,129.5,131.0,...,131.5,48,193,128.5,132.0,170,268,128.0,132.5,119
2019/11/29 13:24,3227,0.0,0,213,130.0,130.5,22,142,129.5,131.0,...,131.5,44,193,128.5,132.0,170,267,128.0,132.5,119
2019/11/29 13:24,3227,0.0,0,213,130.0,130.5,15,142,129.5,131.0,...,131.5,44,183,128.5,132.0,166,268,128.0,132.5,119


In [52]:
#bid ask spread
df['spread1'] = df['Ask1'] - df['Bid1']
df['spread2'] = df['Ask2'] - df['Bid2']
df['spread3'] = df['Ask3'] - df['Bid3']
df['spread4'] = df['Ask4'] - df['Bid4']
df['spread5'] = df['Ask5'] - df['Bid5']
df

,Pid,DealPrice,DealQty,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,...,AskQty4,BidQty5,Bid5,Ask5,AskQty5,spread1,spread2,spread3,spread4,spread5
TxTime,,,,,,,,,,,,,,,,,,,,,
2019/7/1 09:00,3227,126.5,216,60,126.5,127.0,110,78,126.0,127.5,...,30,12,124.5,129.0,68,0.5,1.5,2.5,3.5,4.5
2019/7/1 09:00,3227,127.0,2,60,126.5,127.0,108,78,126.0,127.5,...,30,12,124.5,129.0,68,0.5,1.5,2.5,3.5,4.5
2019/7/1 09:00,3227,127.0,6,62,126.5,127.0,102,79,126.0,127.5,...,30,17,124.5,129.0,73,0.5,1.5,2.5,3.5,4.5
2019/7/1 09:00,3227,127.0,7,63,126.5,127.0,102,78,126.0,127.5,...,30,17,124.5,129.0,74,0.5,1.5,2.5,3.5,4.5
2019/7/1 09:00,3227,127.0,7,93,126.5,127.0,103,78,126.0,127.5,...,29,17,124.5,129.0,74,0.5,1.5,2.5,3.5,4.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019/11/29 13:24,3227,130.5,1,212,130.0,130.5,22,142,129.5,131.0,...,170,268,128.0,132.5,119,0.5,1.5,2.5,3.5,4.5
2019/11/29 13:24,3227,0.0,0,213,130.0,130.5,22,142,129.5,131.0,...,170,267,128.0,132.5,119,0.5,1.5,2.5,3.5,4.5
2019/11/29 13:24,3227,0.0,0,213,130.0,130.5,15,142,129.5,131.0,...,166,268,128.0,132.5,119,0.5,1.5,2.5,3.5,4.5


In [53]:
#midprice
df['mid1'] = (df['Ask1'] + df['Bid1']) / 2
df['mid2'] = (df['Ask2'] + df['Bid2']) / 2
df['mid3'] = (df['Ask3'] + df['Bid3']) / 2
df['mid4'] = (df['Ask4'] + df['Bid4']) / 2
df['mid5'] = (df['Ask5'] + df['Bid5']) / 2
df

,Pid,DealPrice,DealQty,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,...,spread1,spread2,spread3,spread4,spread5,mid1,mid2,mid3,mid4,mid5
TxTime,,,,,,,,,,,,,,,,,,,,,
2019/7/1 09:00,3227,126.5,216,60,126.5,127.0,110,78,126.0,127.5,...,0.5,1.5,2.5,3.5,4.5,126.75,126.75,126.75,126.75,126.75
2019/7/1 09:00,3227,127.0,2,60,126.5,127.0,108,78,126.0,127.5,...,0.5,1.5,2.5,3.5,4.5,126.75,126.75,126.75,126.75,126.75
2019/7/1 09:00,3227,127.0,6,62,126.5,127.0,102,79,126.0,127.5,...,0.5,1.5,2.5,3.5,4.5,126.75,126.75,126.75,126.75,126.75
2019/7/1 09:00,3227,127.0,7,63,126.5,127.0,102,78,126.0,127.5,...,0.5,1.5,2.5,3.5,4.5,126.75,126.75,126.75,126.75,126.75
2019/7/1 09:00,3227,127.0,7,93,126.5,127.0,103,78,126.0,127.5,...,0.5,1.5,2.5,3.5,4.5,126.75,126.75,126.75,126.75,126.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019/11/29 13:24,3227,130.5,1,212,130.0,130.5,22,142,129.5,131.0,...,0.5,1.5,2.5,3.5,4.5,130.25,130.25,130.25,130.25,130.25
2019/11/29 13:24,3227,0.0,0,213,130.0,130.5,22,142,129.5,131.0,...,0.5,1.5,2.5,3.5,4.5,130.25,130.25,130.25,130.25,130.25
2019/11/29 13:24,3227,0.0,0,213,130.0,130.5,15,142,129.5,131.0,...,0.5,1.5,2.5,3.5,4.5,130.25,130.25,130.25,130.25,130.25


In [58]:
#price diff 不小心剪掉了
df

,Pid,DealPrice,DealQty,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,...,a51_diff,b15_diff,a21_diff_abs,a32_diff_abs,a43_diff_abs,a54_diff_abs,b21_diff_abs,b32_diff_abs,b43_diff_abs,b54_diff_abs
TxTime,,,,,,,,,,,,,,,,,,,,,
2019/7/1 09:00,3227,126.5,216,60,126.5,127.0,110,78,126.0,127.5,...,2.0,2.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
2019/7/1 09:00,3227,127.0,2,60,126.5,127.0,108,78,126.0,127.5,...,2.0,2.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
2019/7/1 09:00,3227,127.0,6,62,126.5,127.0,102,79,126.0,127.5,...,2.0,2.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
2019/7/1 09:00,3227,127.0,7,63,126.5,127.0,102,78,126.0,127.5,...,2.0,2.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
2019/7/1 09:00,3227,127.0,7,93,126.5,127.0,103,78,126.0,127.5,...,2.0,2.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019/11/29 13:24,3227,130.5,1,212,130.0,130.5,22,142,129.5,131.0,...,2.0,2.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
2019/11/29 13:24,3227,0.0,0,213,130.0,130.5,22,142,129.5,131.0,...,2.0,2.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
2019/11/29 13:24,3227,0.0,0,213,130.0,130.5,15,142,129.5,131.0,...,2.0,2.0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5


In [59]:
#cross-sectional price and volume's mean
df['a_mean'] = (df['Ask1'] +df['Ask2'] +df['Ask3'] +df['Ask4'] +df['Ask5']) / 5
df['b_mean'] = (df['Bid1'] +df['Bid2'] +df['Bid3'] +df['Bid4'] +df['Bid5']) / 5
df['aq_mean'] = (df['AskQty1'] +df['AskQty2'] +df['AskQty3'] +df['AskQty4'] +df['AskQty5']) / 5
df['bq_mean'] = (df['BidQty1'] +df['BidQty2'] +df['BidQty3'] +df['BidQty4'] +df['BidQty5']) / 5
df

,Pid,DealPrice,DealQty,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,...,a43_diff_abs,a54_diff_abs,b21_diff_abs,b32_diff_abs,b43_diff_abs,b54_diff_abs,a_mean,b_mean,aq_mean,bq_mean
TxTime,,,,,,,,,,,,,,,,,,,,,
2019/7/1 09:00,3227,126.5,216,60,126.5,127.0,110,78,126.0,127.5,...,0.5,0.5,0.5,0.5,0.5,0.5,128.0,125.5,78.6,57.0
2019/7/1 09:00,3227,127.0,2,60,126.5,127.0,108,78,126.0,127.5,...,0.5,0.5,0.5,0.5,0.5,0.5,128.0,125.5,78.2,57.0
2019/7/1 09:00,3227,127.0,6,62,126.5,127.0,102,79,126.0,127.5,...,0.5,0.5,0.5,0.5,0.5,0.5,128.0,125.5,79.0,58.6
2019/7/1 09:00,3227,127.0,7,63,126.5,127.0,102,78,126.0,127.5,...,0.5,0.5,0.5,0.5,0.5,0.5,128.0,125.5,79.0,58.6
2019/7/1 09:00,3227,127.0,7,93,126.5,127.0,103,78,126.0,127.5,...,0.5,0.5,0.5,0.5,0.5,0.5,128.0,125.5,79.2,64.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019/11/29 13:24,3227,130.5,1,212,130.0,130.5,22,142,129.5,131.0,...,0.5,0.5,0.5,0.5,0.5,0.5,131.5,129.0,76.6,226.2
2019/11/29 13:24,3227,0.0,0,213,130.0,130.5,22,142,129.5,131.0,...,0.5,0.5,0.5,0.5,0.5,0.5,131.5,129.0,75.8,226.2
2019/11/29 13:24,3227,0.0,0,213,130.0,130.5,15,142,129.5,131.0,...,0.5,0.5,0.5,0.5,0.5,0.5,131.5,129.0,73.8,223.6


In [60]:
#price and volume's accumulated diff
df['p_acc_diff'] = (df['Ask1'] - df['Bid1']) + (df['Ask1'] - df['Bid1']) +(df['Ask2'] - df['Bid2']) + (df['Ask3'] - df['Bid3']) + (df['Ask4'] - df['Bid4']) + (df['Ask5'] - df['Bid5'])
df['q_acc_diff'] = (df['AskQty1'] - df['BidQty1']) + (df['AskQty1'] - df['BidQty1']) +(df['AskQty2'] - df['BidQty2']) + (df['AskQty3'] - df['BidQty3']) + (df['AskQty4'] - df['BidQty4']) + (df['AskQty5'] - df['BidQty5'])
df

,Pid,DealPrice,DealQty,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,...,b21_diff_abs,b32_diff_abs,b43_diff_abs,b54_diff_abs,a_mean,b_mean,aq_mean,bq_mean,p_acc_diff,q_acc_diff
TxTime,,,,,,,,,,,,,,,,,,,,,
2019/7/1 09:00,3227,126.5,216,60,126.5,127.0,110,78,126.0,127.5,...,0.5,0.5,0.5,0.5,128.0,125.5,78.6,57.0,13.0,158
2019/7/1 09:00,3227,127.0,2,60,126.5,127.0,108,78,126.0,127.5,...,0.5,0.5,0.5,0.5,128.0,125.5,78.2,57.0,13.0,154
2019/7/1 09:00,3227,127.0,6,62,126.5,127.0,102,79,126.0,127.5,...,0.5,0.5,0.5,0.5,128.0,125.5,79.0,58.6,13.0,142
2019/7/1 09:00,3227,127.0,7,63,126.5,127.0,102,78,126.0,127.5,...,0.5,0.5,0.5,0.5,128.0,125.5,79.0,58.6,13.0,141
2019/7/1 09:00,3227,127.0,7,93,126.5,127.0,103,78,126.0,127.5,...,0.5,0.5,0.5,0.5,128.0,125.5,79.2,64.6,13.0,83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019/11/29 13:24,3227,130.5,1,212,130.0,130.5,22,142,129.5,131.0,...,0.5,0.5,0.5,0.5,131.5,129.0,76.6,226.2,13.0,-938
2019/11/29 13:24,3227,0.0,0,213,130.0,130.5,22,142,129.5,131.0,...,0.5,0.5,0.5,0.5,131.5,129.0,75.8,226.2,13.0,-943
2019/11/29 13:24,3227,0.0,0,213,130.0,130.5,15,142,129.5,131.0,...,0.5,0.5,0.5,0.5,131.5,129.0,73.8,223.6,13.0,-947


In [62]:
df['da1_dt'] = df['Ask1'] - df['Ask1'].shift(1)
df['da2_dt'] = df['Ask2'] - df['Ask2'].shift(1)
df['da3_dt'] = df['Ask3'] - df['Ask3'].shift(1)
df['da4_dt'] = df['Ask4'] - df['Ask4'].shift(1)
df['da5_dt'] = df['Ask5'] - df['Ask5'].shift(1)
df['db1_dt'] = df['Bid1'] - df['Bid1'].shift(1)
df['db2_dt'] = df['Bid2'] - df['Bid2'].shift(1)
df['db3_dt'] = df['Bid3'] - df['Bid3'].shift(1)
df['db4_dt'] = df['Bid4'] - df['Bid4'].shift(1)
df['db5_dt'] = df['Bid5'] - df['Bid5'].shift(1)
df['daq1_dt'] = df['AskQty1'] - df['AskQty1'].shift(1)
df['daq2_dt'] = df['AskQty2'] - df['AskQty2'].shift(1)
df['daq3_dt'] = df['AskQty3'] - df['AskQty3'].shift(1)
df['daq4_dt'] = df['AskQty4'] - df['AskQty4'].shift(1)
df['daq5_dt'] = df['AskQty5'] - df['AskQty5'].shift(1)
df['dbq1_dt'] = df['BidQty1'] - df['BidQty1'].shift(1)
df['dbq2_dt'] = df['BidQty2'] - df['BidQty2'].shift(1)
df['dbq3_dt'] = df['BidQty3'] - df['BidQty3'].shift(1)
df['dbq4_dt'] = df['BidQty4'] - df['BidQty4'].shift(1)
df['dbq5_dt'] = df['BidQty5'] - df['BidQty5'].shift(1)
df

,Pid,DealPrice,DealQty,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,...,daq1_dt,daq2_dt,daq3_dt,daq4_dt,daq5_dt,dbq1_dt,dbq2_dt,dbq3_dt,dbq4_dt,dbq5_dt
TxTime,,,,,,,,,,,,,,,,,,,,,
2019/7/1 09:00,3227,126.5,216,60,126.5,127.0,110,78,126.0,127.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019/7/1 09:00,3227,127.0,2,60,126.5,127.0,108,78,126.0,127.5,...,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019/7/1 09:00,3227,127.0,6,62,126.5,127.0,102,79,126.0,127.5,...,-6.0,5.0,0.0,0.0,5.0,2.0,1.0,0.0,0.0,5.0
2019/7/1 09:00,3227,127.0,7,63,126.5,127.0,102,78,126.0,127.5,...,0.0,-1.0,0.0,0.0,1.0,1.0,-1.0,0.0,0.0,0.0
2019/7/1 09:00,3227,127.0,7,93,126.5,127.0,103,78,126.0,127.5,...,1.0,1.0,0.0,-1.0,0.0,30.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019/11/29 13:24,3227,130.5,1,212,130.0,130.5,22,142,129.5,131.0,...,-1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2019/11/29 13:24,3227,0.0,0,213,130.0,130.5,22,142,129.5,131.0,...,0.0,0.0,-4.0,0.0,0.0,1.0,0.0,0.0,0.0,-1.0
2019/11/29 13:24,3227,0.0,0,213,130.0,130.5,15,142,129.5,131.0,...,-7.0,1.0,0.0,-4.0,0.0,0.0,0.0,-4.0,-10.0,1.0


In [66]:
##pred if 今天 > 前10天平均
df['m_b'] = df['mid1'].rolling(10).mean()
df

,Pid,DealPrice,DealQty,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,...,daq2_dt,daq3_dt,daq4_dt,daq5_dt,dbq1_dt,dbq2_dt,dbq3_dt,dbq4_dt,dbq5_dt,m_b
TxTime,,,,,,,,,,,,,,,,,,,,,
2019/7/1 09:00,3227,126.5,216,60,126.5,127.0,110,78,126.0,127.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019/7/1 09:00,3227,127.0,2,60,126.5,127.0,108,78,126.0,127.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2019/7/1 09:00,3227,127.0,6,62,126.5,127.0,102,79,126.0,127.5,...,5.0,0.0,0.0,5.0,2.0,1.0,0.0,0.0,5.0,NaN
2019/7/1 09:00,3227,127.0,7,63,126.5,127.0,102,78,126.0,127.5,...,-1.0,0.0,0.0,1.0,1.0,-1.0,0.0,0.0,0.0,NaN
2019/7/1 09:00,3227,127.0,7,93,126.5,127.0,103,78,126.0,127.5,...,1.0,0.0,-1.0,0.0,30.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019/11/29 13:24,3227,130.5,1,212,130.0,130.5,22,142,129.5,131.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,130.25
2019/11/29 13:24,3227,0.0,0,213,130.0,130.5,22,142,129.5,131.0,...,0.0,-4.0,0.0,0.0,1.0,0.0,0.0,0.0,-1.0,130.25
2019/11/29 13:24,3227,0.0,0,213,130.0,130.5,15,142,129.5,131.0,...,1.0,0.0,-4.0,0.0,0.0,0.0,-4.0,-10.0,1.0,130.25


In [67]:
df.head(20)

,Pid,DealPrice,DealQty,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,...,daq2_dt,daq3_dt,daq4_dt,daq5_dt,dbq1_dt,dbq2_dt,dbq3_dt,dbq4_dt,dbq5_dt,m_b
TxTime,,,,,,,,,,,,,,,,,,,,,
2019/7/1 09:00,3227,126.5,216,60,126.5,127.0,110,78,126.0,127.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019/7/1 09:00,3227,127.0,2,60,126.5,127.0,108,78,126.0,127.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2019/7/1 09:00,3227,127.0,6,62,126.5,127.0,102,79,126.0,127.5,...,5.0,0.0,0.0,5.0,2.0,1.0,0.0,0.0,5.0,NaN
2019/7/1 09:00,3227,127.0,7,63,126.5,127.0,102,78,126.0,127.5,...,-1.0,0.0,0.0,1.0,1.0,-1.0,0.0,0.0,0.0,NaN
2019/7/1 09:00,3227,127.0,7,93,126.5,127.0,103,78,126.0,127.5,...,1.0,0.0,-1.0,0.0,30.0,0.0,0.0,0.0,0.0,NaN
2019/7/1 09:01,3227,127.0,102,92,126.5,127.0,8,78,126.0,127.5,...,0.0,0.0,0.0,-1.0,-1.0,0.0,6.0,1.0,0.0,NaN
2019/7/1 09:01,3227,127.0,11,142,127.0,127.5,40,93,126.5,128.0,...,109.0,-120.0,44.0,-63.0,50.0,15.0,19.0,-24.0,67.0,NaN
2019/7/1 09:01,3227,127.0,10,139,127.0,127.5,44,70,126.5,128.0,...,3.0,-3.0,-5.0,4.0,-3.0,-23.0,2.0,4.0,-2.0,NaN
2019/7/1 09:01,3227,127.0,10,139,127.0,127.5,44,62,126.5,128.0,...,1.0,0.0,0.0,0.0,0.0,-8.0,0.0,-2.0,2.0,NaN


In [75]:
df['pred1'] = df['m_b']
for i in range(len(df) - 1):
    if df['mid1'].iloc[i+1] > df['m_b'].iloc[i] * (1 + alpha[0]):
        df['pred1'].iloc[i+1] = 1
    elif df['mid1'].iloc[i+1] < df['m_b'].iloc[i] * (1 - alpha[0]):
        df['pred1'].iloc[i+1] = -1
    else:
        df['pred1'].iloc[i+1] = 0

In [85]:
df

,Pid,DealPrice,DealQty,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,...,daq3_dt,daq4_dt,daq5_dt,dbq1_dt,dbq2_dt,dbq3_dt,dbq4_dt,dbq5_dt,m_b,pred1
TxTime,,,,,,,,,,,,,,,,,,,,,
2019/7/1 09:00,3227,126.5,216,60,126.5,127.0,110,78,126.0,127.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019/7/1 09:00,3227,127.0,2,60,126.5,127.0,108,78,126.0,127.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0
2019/7/1 09:00,3227,127.0,6,62,126.5,127.0,102,79,126.0,127.5,...,0.0,0.0,5.0,2.0,1.0,0.0,0.0,5.0,NaN,0.0
2019/7/1 09:00,3227,127.0,7,63,126.5,127.0,102,78,126.0,127.5,...,0.0,0.0,1.0,1.0,-1.0,0.0,0.0,0.0,NaN,0.0
2019/7/1 09:00,3227,127.0,7,93,126.5,127.0,103,78,126.0,127.5,...,0.0,-1.0,0.0,30.0,0.0,0.0,0.0,0.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019/11/29 13:24,3227,130.5,1,212,130.0,130.5,22,142,129.5,131.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,130.25,0.0
2019/11/29 13:24,3227,0.0,0,213,130.0,130.5,22,142,129.5,131.0,...,-4.0,0.0,0.0,1.0,0.0,0.0,0.0,-1.0,130.25,0.0
2019/11/29 13:24,3227,0.0,0,213,130.0,130.5,15,142,129.5,131.0,...,0.0,-4.0,0.0,0.0,0.0,-4.0,-10.0,1.0,130.25,0.0


In [77]:
df['pred1'].count()

253059

In [83]:
df.pred1.value_counts()

 0.0    192309
-1.0     30391
 1.0     30359
Name: pred1, dtype: int64

In [86]:
df.to_csv('./data/data_2.csv',index=False)

In [94]:
factor = df.columns

In [92]:
df = df.drop(['Pid', 'DealPrice', 'DealQty', 'm_b', 'pred1'], axis=1)
df

,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,AskQty2,BidQty3,Bid3,...,daq1_dt,daq2_dt,daq3_dt,daq4_dt,daq5_dt,dbq1_dt,dbq2_dt,dbq3_dt,dbq4_dt,dbq5_dt
TxTime,,,,,,,,,,,,,,,,,,,,,
2019/7/1 09:00,60,126.5,127.0,110,78,126.0,127.5,36,53,125.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019/7/1 09:00,60,126.5,127.0,108,78,126.0,127.5,36,53,125.5,...,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019/7/1 09:00,62,126.5,127.0,102,79,126.0,127.5,41,53,125.5,...,-6.0,5.0,0.0,0.0,5.0,2.0,1.0,0.0,0.0,5.0
2019/7/1 09:00,63,126.5,127.0,102,78,126.0,127.5,40,53,125.5,...,0.0,-1.0,0.0,0.0,1.0,1.0,-1.0,0.0,0.0,0.0
2019/7/1 09:00,93,126.5,127.0,103,78,126.0,127.5,41,53,125.5,...,1.0,1.0,0.0,-1.0,0.0,30.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019/11/29 13:24,212,130.0,130.5,22,142,129.5,131.0,24,316,129.0,...,-1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2019/11/29 13:24,213,130.0,130.5,22,142,129.5,131.0,24,316,129.0,...,0.0,0.0,-4.0,0.0,0.0,1.0,0.0,0.0,0.0,-1.0
2019/11/29 13:24,213,130.0,130.5,15,142,129.5,131.0,25,312,129.0,...,-7.0,1.0,0.0,-4.0,0.0,0.0,0.0,-4.0,-10.0,1.0


In [93]:
df.to_csv('./data/data_3.csv',index=False)

In [100]:
df['mid1_t+1'] = df['mid1'].shift(-1)
df = df.drop(['mid1'], axis = 1)
df

,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,AskQty2,BidQty3,Bid3,...,daq2_dt,daq3_dt,daq4_dt,daq5_dt,dbq1_dt,dbq2_dt,dbq3_dt,dbq4_dt,dbq5_dt,mid1_t+1
TxTime,,,,,,,,,,,,,,,,,,,,,
2019/7/1 09:00,60,126.5,127.0,110,78,126.0,127.5,36,53,125.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,126.75
2019/7/1 09:00,60,126.5,127.0,108,78,126.0,127.5,36,53,125.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,126.75
2019/7/1 09:00,62,126.5,127.0,102,79,126.0,127.5,41,53,125.5,...,5.0,0.0,0.0,5.0,2.0,1.0,0.0,0.0,5.0,126.75
2019/7/1 09:00,63,126.5,127.0,102,78,126.0,127.5,40,53,125.5,...,-1.0,0.0,0.0,1.0,1.0,-1.0,0.0,0.0,0.0,126.75
2019/7/1 09:00,93,126.5,127.0,103,78,126.0,127.5,41,53,125.5,...,1.0,0.0,-1.0,0.0,30.0,0.0,0.0,0.0,0.0,126.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019/11/29 13:24,212,130.0,130.5,22,142,129.5,131.0,24,316,129.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,130.25
2019/11/29 13:24,213,130.0,130.5,22,142,129.5,131.0,24,316,129.0,...,0.0,-4.0,0.0,0.0,1.0,0.0,0.0,0.0,-1.0,130.25
2019/11/29 13:24,213,130.0,130.5,15,142,129.5,131.0,25,312,129.0,...,1.0,0.0,-4.0,0.0,0.0,0.0,-4.0,-10.0,1.0,130.25


In [102]:
df.to_csv('./data/3227_t+1.csv')

In [103]:
df['next_mid1'] = df['mid1_t+1']
df = df.drop(['mid1_t+1'], axis = 1)
df

,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,AskQty2,BidQty3,Bid3,...,daq2_dt,daq3_dt,daq4_dt,daq5_dt,dbq1_dt,dbq2_dt,dbq3_dt,dbq4_dt,dbq5_dt,next_mid1
TxTime,,,,,,,,,,,,,,,,,,,,,
2019/7/1 09:00,60,126.5,127.0,110,78,126.0,127.5,36,53,125.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,126.75
2019/7/1 09:00,60,126.5,127.0,108,78,126.0,127.5,36,53,125.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,126.75
2019/7/1 09:00,62,126.5,127.0,102,79,126.0,127.5,41,53,125.5,...,5.0,0.0,0.0,5.0,2.0,1.0,0.0,0.0,5.0,126.75
2019/7/1 09:00,63,126.5,127.0,102,78,126.0,127.5,40,53,125.5,...,-1.0,0.0,0.0,1.0,1.0,-1.0,0.0,0.0,0.0,126.75
2019/7/1 09:00,93,126.5,127.0,103,78,126.0,127.5,41,53,125.5,...,1.0,0.0,-1.0,0.0,30.0,0.0,0.0,0.0,0.0,126.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019/11/29 13:24,212,130.0,130.5,22,142,129.5,131.0,24,316,129.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,130.25
2019/11/29 13:24,213,130.0,130.5,22,142,129.5,131.0,24,316,129.0,...,0.0,-4.0,0.0,0.0,1.0,0.0,0.0,0.0,-1.0,130.25
2019/11/29 13:24,213,130.0,130.5,15,142,129.5,131.0,25,312,129.0,...,1.0,0.0,-4.0,0.0,0.0,0.0,-4.0,-10.0,1.0,130.25


In [104]:
df.to_csv('./data/3227set.csv')

In [107]:
df['Bid1'].iloc[0:10, ]

TxTime
2019/7/1 09:00    126.5
2019/7/1 09:00    126.5
2019/7/1 09:00    126.5
2019/7/1 09:00    126.5
2019/7/1 09:00    126.5
2019/7/1 09:01    126.5
2019/7/1 09:01    127.0
2019/7/1 09:01    127.0
2019/7/1 09:01    127.0
2019/7/1 09:01    127.0
Name: Bid1, dtype: float64

In [108]:
df.head(10)

,BidQty1,Bid1,Ask1,AskQty1,BidQty2,Bid2,Ask2,AskQty2,BidQty3,Bid3,...,daq2_dt,daq3_dt,daq4_dt,daq5_dt,dbq1_dt,dbq2_dt,dbq3_dt,dbq4_dt,dbq5_dt,next_mid1
TxTime,,,,,,,,,,,,,,,,,,,,,
2019/7/1 09:00,60,126.5,127.0,110,78,126.0,127.5,36,53,125.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,126.75
2019/7/1 09:00,60,126.5,127.0,108,78,126.0,127.5,36,53,125.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,126.75
2019/7/1 09:00,62,126.5,127.0,102,79,126.0,127.5,41,53,125.5,...,5.0,0.0,0.0,5.0,2.0,1.0,0.0,0.0,5.0,126.75
2019/7/1 09:00,63,126.5,127.0,102,78,126.0,127.5,40,53,125.5,...,-1.0,0.0,0.0,1.0,1.0,-1.0,0.0,0.0,0.0,126.75
2019/7/1 09:00,93,126.5,127.0,103,78,126.0,127.5,41,53,125.5,...,1.0,0.0,-1.0,0.0,30.0,0.0,0.0,0.0,0.0,126.75
2019/7/1 09:01,92,126.5,127.0,8,78,126.0,127.5,41,59,125.5,...,0.0,0.0,0.0,-1.0,-1.0,0.0,6.0,1.0,0.0,127.25
2019/7/1 09:01,142,127.0,127.5,40,93,126.5,128.0,150,78,126.0,...,109.0,-120.0,44.0,-63.0,50.0,15.0,19.0,-24.0,67.0,127.25
2019/7/1 09:01,139,127.0,127.5,44,70,126.5,128.0,153,80,126.0,...,3.0,-3.0,-5.0,4.0,-3.0,-23.0,2.0,4.0,-2.0,127.25
2019/7/1 09:01,139,127.0,127.5,44,62,126.5,128.0,154,80,126.0,...,1.0,0.0,0.0,0.0,0.0,-8.0,0.0,-2.0,2.0,127.25
